In [1]:
#모든 환자가 다 같도록 만들고
#데이터가 한 쪽이 더 많지 않도록 균형 맞추기
#1. Normal vs Abnormal: 258개로 통일
#Normal 291, Abnormal(MCI 129, AD 129) 구성
#2. Normal vs AD: 208개로 통일
#Normal 208, AD 208
#3. Normal vs MCI: 228개로 통일
#Normal 228, MCI 228
#4. MCI vs AD: 208개로 통일
#MCI 208, AD 208

#현재 문항별 보유 데이터: data_distribution_중복제거_모든응답을한사람만.xls



In [2]:

import os
import pandas as pd
import numpy as np
import shutil #file copy

filePathArr=[]
durationArr=[]
fileNameArr=[]



In [3]:
base_path = './temp/'
versionname = 'v5'
#디렉토리를 생성할 때마다 랜덤으로 파일이 선택됩니다. versionname 폴더명을 변경하세요.

In [4]:
#5fold, iteration folder 만들기
for classifi in ['MCI_AD','normal_abnormal','SCI_AD','SCI_MCI']:
    nab = False
    for qnum in range(1, 12):
        
        os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/", exist_ok=True, mode=0o777)
        if classifi =="MCI_AD":
            class1 = "MCI"
            class2 = "AD"
        elif classifi == "normal_abnormal":
            class1 = "normal"
            class2 = "abnormal"
            nab = True
            class3 = "MCI"
            class4 = "AD"
        elif classifi == "SCI_AD":
            class1 = "SCI"
            class2 = "AD"
        elif classifi =="SCI_MCI":
            class1 = "SCI"
            class2 = "MCI"
        os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/"+class1+"/", exist_ok=True, mode=0o777)
        os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/"+class2+"/", exist_ok=True, mode=0o777)
        #os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/test/"+class1+"/", exist_ok=True, mode=0o777)
        #os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/test/"+class2+"/", exist_ok=True, mode=0o777)
        #if nab == True:
        #    os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/"+class2+"/"+class3+"/", exist_ok=True, mode=0o777)
        #    os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/test/"+class2+"/"+class3+"/", exist_ok=True, mode=0o777)
        #    os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/"+class2+"/"+class4+"/", exist_ok=True, mode=0o777)
        #    os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/test/"+class2+"/"+class4+"/", exist_ok=True, mode=0o777)

        
        #if classifi == "normal_abnormal":
        #    os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/abnormal/MCI/", exist_ok=True, mode=0o777)
        #    os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/abnormal/AD", exist_ok=True, mode=0o777)
        #    #os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/test/abnormal/AD", exist_ok=True)
        #    #os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/test/abnormal/MCI", exist_ok=True)

        for iternum in range(1,6):
            for tt in ['test','train']:
                os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/iterations/iteration"+str(iternum)+"/"+tt+"/"+class1+"/", exist_ok=True, mode=0o777)
                os.makedirs(base_path+versionname+"/"+classifi+"/"+str(qnum)+"/iterations/iteration"+str(iternum)+"/"+tt+"/"+class2+"/", exist_ok=True, mode=0o777)


In [5]:

#clear save_path : 새로 저장할 때 다시 random으로 추출해서 파일을 저장하므로 기존 파일 내용 삭제
"""
save_path = ['./temp/'+versionname+'/normal_abnormal/']

for dir in save_path:
    for num in range(1,12):
        for disease in ('normal', 'abnormal'):
            file_path = dir + str(num) + '/' + disease + '/'
            file_list = os.listdir(file_path)
            
            for k in range(0, len(file_list)):
                os.remove(file_path+file_list[k])
    
"""

"\nsave_path = ['./temp/'+versionname+'/normal_abnormal/']\n\nfor dir in save_path:\n    for num in range(1,12):\n        for disease in ('normal', 'abnormal'):\n            file_path = dir + str(num) + '/' + disease + '/'\n            file_list = os.listdir(file_path)\n            \n            for k in range(0, len(file_list)):\n                os.remove(file_path+file_list[k])\n    \n"

In [6]:
#여기에서 파일을 복사해 옴. 균형을 맞추고, 모두 응답한 사람들만 남겨둔 ALL_pure 디렉토리에서 뽑기.
#os.makedirs('./temp/ALL_pure')
base_path = ['./temp/ALL_pure/']

pick_no = 129
disease='MCI'

#MCI=146개 비복원 추출
file_path = base_path[0]+ str(1) + '/' + disease + '/' 
file_list = os.listdir(file_path)
#print(len(file_list))
pick=np.random.choice(len(file_list), pick_no, replace=False)#뽑는 환자들 id는 랜덤으로 뽑되, 모든 문항에서 고정되어 있어야 함.
print(pick)
print(len(pick))
#print(file_list)

uid = []
for item in file_list:
    if len(item)<13:#nia
        uid.append(item[:-6]) 
    else:
        uid.append(item[:-8])
        
#print(uid)

for num in range(1,12):
    file_path = base_path[0]+ str(num) + '/' + disease + '/' 
    
    uid_qnum = []
    
    for item in uid:
        if len(item)<7:
            uid_qnum.append(f'{item}.{num}.png')
        else:
            uid_qnum.append(f'{item}_{num-1}_R.png')
            
    #print("uid_qnum",uid_qnum)

    for dir in base_path:

        save_path = './temp/'+versionname+'/normal_abnormal/'+str(num)+'/abnormal/'
        file_path = base_path[0]+ str(num) + '/' + disease + '/' 
        file_list = os.listdir(file_path)

        for k in pick:
            shutil.copy(file_path+uid_qnum[k], save_path)

            
            

[  5 229 210 132 150 145 177 203  60  72   4  89  56 201 217 146  83  92
  48 130 166  32 189 205 108 219  71 214  85  36  90 197  68  10  49 149
 224  57 117  82  55  50 220 161 152 225 102 173 111 115  88 175 198 218
  17 121 162 107  31  94  96  86  63 193  80 213  20 208   1 138 191  11
  13  25 199  19 100 106  28 142 103  53 211 196 144  62   6 221  97 170
  66 129 204 207  76 159 192 155 165 164  75 172  98 147  77 131 110  37
 226  69 133  14 124  40  95 190 181 140 168 153 143 174 127  51 112  45
 212 180 134]
129


In [7]:
#여기에서 파일을 복사해 옴. 균형을 맞추고, 모두 응답한 사람들만 남겨둔 ALL_pure 디렉토리에서 뽑기.
#os.makedirs('./temp/ALL_pure')
base_path = ['./temp/ALL_pure/']

pick_no = 129
disease='AD'

#MCI=146개 비복원 추출
file_path = base_path[0]+ str(1) + '/' + disease + '/' 
file_list = os.listdir(file_path)
#print(len(file_list))
pick=np.random.choice(len(file_list), pick_no, replace=False)#뽑는 환자들 id는 랜덤으로 뽑되, 모든 문항에서 고정되어 있어야 함.
print(pick)
print(len(pick))
#print(file_list)

uid = []
for item in file_list:
    if len(item)<13:#nia
        uid.append(item[:-6]) 
    else:
        uid.append(item[:-8])
        
#print(uid)

for num in range(1,12):
    file_path = base_path[0]+ str(num) + '/' + disease + '/' 
    
    uid_qnum = []
    
    for item in uid:
        if len(item)<7:
            uid_qnum.append(f'{item}.{num}.png')
        else:
            uid_qnum.append(f'{item}_{num-1}_R.png')
            
    #print("uid_qnum",uid_qnum)

    for dir in base_path:

        save_path = './temp/'+versionname+'/normal_abnormal/'+str(num)+'/abnormal/'
        file_path = base_path[0]+ str(num) + '/' + disease + '/' 
        file_list = os.listdir(file_path)

        for k in pick:
            shutil.copy(file_path+uid_qnum[k], save_path)

            
            

[131  47 132 204  62 200 115 163 118  72  31 157  46  50  70  60 166  13
  28  16 194  48 208  35   3 183  17 136 117 151  98  63 116  23 147 135
  19  91  12 149 111 139 180  75  34 196  64  99  81 125  90 167 122 143
   9  21 178 155  29  74 104 110 120   8  94  39  87 112 150  79 182  27
  15  32 107   2  83 156 103  57  78 173  45   7  10  86  36  67 162  59
  44 158  92 137  58 174 128  38 154 159 197  25 168  65 124 138 171  40
 160 187 109 146 172  56 207  18  24  85 195  80 192  68 202  20  96 206
 134   0 165]
129


In [8]:
#여기에서 파일을 복사해 옴. 균형을 맞추고, 모두 응답한 사람들만 남겨둔 ALL_pure 디렉토리에서 뽑기.
#os.makedirs('./temp/ALL_pure')
base_path = ['./temp/ALL_pure/']

pick_no = 258
disease='SCI'

#MCI=146개 비복원 추출
file_path = base_path[0]+ str(1) + '/' + disease + '/' 
file_list = os.listdir(file_path)
#print(len(file_list))
pick=np.random.choice(len(file_list), pick_no, replace=False)#뽑는 환자들 id는 랜덤으로 뽑되, 모든 문항에서 고정되어 있어야 함.
print(pick)
print(len(pick))
#print(file_list)

uid = []
for item in file_list:
    if len(item)<13:#nia
        uid.append(item[:-6]) 
    else:
        uid.append(item[:-8])
        
#print(uid)

for num in range(1,12):
    file_path = base_path[0]+ str(num) + '/' + disease + '/' 
    
    uid_qnum = []
    
    for item in uid:
        if len(item)<7:
            uid_qnum.append(f'{item}.{num}.png')
        else:
            uid_qnum.append(f'{item}_{num-1}_R.png')
            
    #print("uid_qnum",uid_qnum)

    for dir in base_path:

        save_path = './temp/'+versionname+'/normal_abnormal/'+str(num)+'/normal/'
        file_path = base_path[0]+ str(num) + '/' + disease + '/' 
        file_list = os.listdir(file_path)

        for k in pick:
            shutil.copy(file_path+uid_qnum[k], save_path)

            
            

[238 107  99 181  77 205 185 240  10 179 187   7 224   2 227  96  95   1
 249  23 124 128 137 199 139 219 190 245 122 200 256 160 184 117 129 125
  90  36  83 188  18 239  67 108 209  19 168   4 106 233 114 208 207 161
  76  24  34 202  28   0 155  16  98  40 253 118 152 148 136  81 103 126
 111 220  21 142 149 140  86  54 183 222 171  14  35  50  45  91  48  46
 221 193 119 192 225 177  72  82 176 243 252  62 247 109 254 162  32  52
 157  20   8 132 150  12 131 154 211   5 191 100  31  15 174 147 116 257
  30  79  58  70   9  43 236  84   3  11 104 212 196 110  78 255 198 115
  25  73 173 244 210 127 172 251  57 215 164 123 250 218  55 144 120 178
  69 186 145  75 101 246 102  64 159 217 112 165 138  33 213 231  49  80
 166 133  66 226 234 232 194  53 143  26  88  41 242  71 230 153 203  92
  44 204  39  61  13  17  47  89  68 180 197 146  74  29 163  87 235  59
 151  63  93  51  97 167 223 195 228   6 169 156 201 214 216 141  38 182
 121 130  60 158 241  37 237  42 248  65 135 189 17